<a href="https://colab.research.google.com/github/Manu-Sanchez/Learning/blob/ai%2Fbasic-gan/Custom_Images_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as func
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
root = "../data"

train_data = datasets.ImageFolder(os.path.join(root, 'train'), transforms=train_transforms)
test_data = datasets.ImageFolder(os.path.join(root, 'test'), transforms=test_transforms)

torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_data = DataLoader(test_data, batch_size=10, shuffle=False)

class_names = train_data.classes #Cat, Dog

len(train_data)#18743
len(test_data)#6251

In [ ]:
for img_batch, label_batch in train_loader:
  break

img_batch.shape#10, 3, 224 224

In [ ]:
im = make_grid(img_batch, nrows=5)

inv_normalize = transforms.Normalice(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

img_inv = inv_normalize(im)
plt.imshow(im_inv.permute(1,2,0))

In [ ]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    self.fc1 = nn.Linear(54*54*16, 120) # ((224 - 2)//2) = ((111 - 2)) // 2 = 54
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 2)

  def forward(self, x):
    x = func.relu(self.conv1(x))
    x = func.max_pool2d(x, 2, 2)
    x = func.relu(self.conv2(x))
    x = func.max_pool2d(x, 2, 2)
    x = func.relu(self.fc1(x.reshape(-1, 54*54*16)))
    x = func.relu(self.fc2(x))
    x = self.fc3(x)

    return func.log_softmax(x, dim=1)

In [ ]:
model = ConvolutionalNetwork()
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
model

In [ ]:
for param in model.parameters():
  print(param.numel())

In [ ]:
epochs = 3

max_trn_batch = 800
max_tst_batch = 300

train_loss = []
test_loss = []

train_correct = []
test_correct = []

for i in range(epochs):
  train_hits = 0
  test_hits = 0

  for b, (x_tr, y_tr) in enumerate(train_loader):
    if b == max_trn_batch: break

    y_pred = model(x_tr)
    loss = criterion(y_pred, y_tr)

    optim.zero_grad()
    loss.backward()
    optim.step()

    hits = (torch.max(y_pred, 1)[1] == y_tr).sum()
    train_hits += hits

    if b%200 == 0:
      print(f"Epoch {i} Batch {b} Loss {loss.item()}")

  train_loss.append(loss)
  train_correct.append(hits)

  with torch.no_grad():
    for b, (x_ts, y_ts) in enumerate(test_loader):
      if b == max_tst_batch: break

      y_pred = model(x_ts)

      hits = (torch.max(y_pred, 1)[1] == y_ts).sum()
      test_hits += hits

  loss = criterion(y_pred, y_tr)
  test_loss.append(loss)
  test_correct.append(test_hits)


In [ ]:
torch.save(model.state_dict(), "model.pt")